# RecSys via Clustering

Using the IMDB and MovieLens data, we build a feature matrix that we can use for recommendation.

We build clusters of users from this data and evaluate the most popular movies in each cluster as our recommendations.

In [1]:
%matplotlib inline

In [2]:
import json
import random
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from collections import Counter

from scipy.sparse import lil_matrix

from sklearn.neighbors import DistanceMetric

from sklearn.metrics import jaccard_score
from sklearn.metrics import pairwise_distances

In [3]:
known_movies = set()

user_ratings = {} # List of all our movie ratings for specific users
movie_ids = []

with open("../data/user_ratings.json", "r") as in_file:
    for line in in_file:
        
        this_rating = json.loads(line)
        
        known_movies.add(this_rating["title_id"])
        
        if this_rating["title_id"] not in movie_ids:
            movie_ids.append(this_rating["title_id"])
        
        this_users_ratings = user_ratings.get(this_rating["userId"], [])
        this_users_ratings.append((this_rating["title_id"], this_rating["rating"]))
        
        user_ratings[this_rating["userId"]] = this_users_ratings
        
        

In [4]:
movie_id_to_index = {m:i for i,m in enumerate(movie_ids)}

In [5]:
print("Known Users:", len(user_ratings))
print("Known Movies:", len(known_movies))


Known Users: 2244
Known Movies: 4465


In [6]:
actor_id_to_name_map = {}     # Map Actor IDs to actor names
actor_id_to_index_map = {}    # Map actor IDs to a unique index of known actors
index_to_actor_ids = []       # Array mapping unique index back to actor ID (invert of actor_id_to_index_map)

index_counter = 0    # Unique actor index; increment for each new actor
known_actors = set()

movie_actor_map = {} # List of all our movies and their actors

test_count = 0
with open("../data/imdb_recent_movies.json", "r") as in_file:
    for line in in_file:
        
        this_movie = json.loads(line)
        
        # Restrict to known movies
        if this_movie["title_id"] not in known_movies:
            continue
            
        # Keep track of all the actors in this movie
        for actor_id,actor_name in zip(this_movie['actor_ids'],this_movie['actor_names']):
            
            # Keep names and IDs
            actor_id_to_name_map[actor_id] = actor_name
            
            # If we've seen this actor before, skip...
            if actor_id in known_actors:
                continue
                
            # ... Otherwise, add to known actor set and create new index for them
            known_actors.add(actor_id)
            actor_id_to_index_map[actor_id] = index_counter
            index_to_actor_ids.append(actor_id)
            index_counter += 1
            
        # Finished with this film
        movie_actor_map[this_movie["title_id"]] = ({
            "movie": this_movie["title_name"],
            "actors": set(this_movie['actor_ids']),
            "genres": this_movie["title_genre"]
        })

In [7]:
print("Known Actors:", len(known_actors))
print("Known Movies:", len(movie_actor_map))

Known Actors: 5224
Known Movies: 4465


## Generate DataFrame using Sparse Matrics

Convert our Movie Ratings data into a DataFrame that we can use for analysis.

In [8]:
# With sparse matrix, initialize to size of Users x Movies of 0s
matrix_sparse = lil_matrix((len(user_ratings), len(known_movies)), dtype=float)

# Update the matrix, user by user, setting non-zero values for the appropriate actors
for row,this_user in enumerate(user_ratings): 
    this_user_ratings = user_ratings[this_user]
    
    for movie_id,rating in this_user_ratings:
        this_movie_index = movie_id_to_index[movie_id]
        matrix_sparse[row,this_movie_index] = rating

In [9]:
df = pd.DataFrame.sparse.from_spmatrix(
    matrix_sparse, 
    index=[u for u in user_ratings],
    columns=movie_ids
).T
df

,10,37,51,126,152,263,284,448,626,706,...,162002,162073,162207,162257,162363,162420,162434,162464,162499,162537
tt0274309,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0298203,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0315733,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0337563,1.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0463854,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt4241904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
tt1666800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
tt6806448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0844671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [10]:
print("Movies x Users dimensionality:", df.shape)

Movies x Users dimensionality: (4465, 2244)


In [11]:
from sklearn.cluster import KMeans
from numpy.linalg import norm

In [12]:
kmeans = KMeans(n_clusters=4, random_state=42).fit(df.T)

In [100]:
new_df = df.T

In [102]:
new_df['predicted_cluster'] = list(kmeans.labels_)

In [103]:
new_df.predicted_cluster.value_counts()

2    1635
0     311
3     225
1      74
Name: predicted_cluster, dtype: int64

In [148]:
user_series = new_df.loc["126"][:-1]
not_watched_series = user_series[user_series.values == 0]
"tt0209144" in not_watched_series.index

True

In [157]:
user_series = new_df.loc["126"][:-1]
watched_series = user_series[user_series.values > 0]
seen_movies = list(watched_series.index)

In [106]:
group_2 = new_df[new_df['predicted_cluster'] == 2]#.drop(['126'], inplace=True)

In [107]:
group_2.drop(['126'], inplace = True)

C:\Users\dhlec\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [115]:
group_2.drop(columns = ['predicted_cluster'], inplace = True)

C:\Users\dhlec\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [158]:
group_2.drop(columns = seen_movies, inplace = True)

C:\Users\dhlec\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [161]:
group_2[group_2 > 0].sum(axis = 0).sort_values(ascending = False).head(20)#/group_2[group_2 > 0].count(axis = 0).sort_values(ascending = False).head(20)

tt0209144    1618.0
tt0126029    1297.5
tt0338013    1169.0
tt0198781     953.5
tt0266543     926.5
tt0910970     893.0
tt0120903     863.0
tt0266697     850.0
tt1049413     776.0
tt0246578     774.0
tt0162222     764.5
tt0372784     740.0
tt0190590     723.5
tt0181689     718.5
tt0240772     708.5
tt0816692     688.0
tt0378194     686.5
tt0181875     676.0
tt0208092     670.0
tt0264464     637.5
dtype: float64

In [162]:
for i in list(group_2[group_2 > 0].count(axis = 0).sort_values(ascending = False).head(20).index):
    print(movie_actor_map[i]['movie'])

Memento
Shrek
Eternal Sunshine of the Spotless Mind
Monsters, Inc.
X-Men
Finding Nemo
WALL·E
Kill Bill: Vol. 1
Cast Away
Minority Report
Donnie Darko
Batman Begins
Up
Ocean's Eleven
O Brother, Where Art Thou?
Kill Bill: Vol. 2
Almost Famous
Spider-Man
Erin Brockovich
Interstellar


In [132]:
group_2[group_2 > 0].sum().sum()/group_2[group_2 > 0].count().sum()

3.5948767900475067

In [67]:
cluster_series = new_df[new_df['predicted_cluster'] == 2].max().sort_values(ascending = False)
A

tt4263482    5.0
tt1611845    5.0
tt0804522    5.0
tt1392170    5.0
tt0945513    5.0
            ... 
tt3741700    0.0
tt0854678    0.0
tt3171832    0.0
tt3095734    0.0
tt1727770    0.0
Length: 4466, dtype: float64

In [36]:
group_0 = df[df['predicted_cluster'] == 0][list(df.columns[:-1])]
group_1 = df[df['predicted_cluster'] == 1][list(df.columns[:-1])]

In [64]:
cosine = np.dot(A,B)/(norm(A)*norm(B))

In [65]:
cosine

0.08713217330464067